# Reshaping, Reorganizing and Aggregation

In [1]:
# import pandas and NumPy
import pandas as pd
import numpy as np

# date and time functions
import datetime

# bring in matplotlib and draw inline
import matplotlib.pyplot as plt
%matplotlib inline

# Set some Pandas options
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 8)
pd.set_option('precision', 3)

## Loading historical stock data from the web or from files

### From Web

In [43]:
import yfinance as yf
import datetime

# Start and end dates
start = datetime.datetime(2012, 1, 1)
end = datetime.datetime(2012, 12, 30)

# Load the data
msft = yf.download("MSFT", start=start, end=end)
aapl = yf.download("AAPL", start=start, end=end)

# Optional: Save the data to file
msft.to_csv("msft.csv")
aapl.to_csv("aapl.csv")

# Display the first few rows of each DataFrame
print(msft.head())
print(aapl.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

             Open   High    Low  Close  Adj Close    Volume
Date                                                       
2012-01-03  26.55  26.96  26.39  26.77     21.201  64731500
2012-01-04  26.82  27.47  26.78  27.40     21.699  80516100
2012-01-05  27.38  27.73  27.29  27.68     21.921  56081400
2012-01-06  27.53  28.19  27.53  28.11     22.262  99455500
2012-01-09  28.05  28.10  27.72  27.74     21.969  59706800
              Open    High     Low   Close  Adj Close     Volume
Date                                                            
2012-01-03  14.621  14.732  14.607  14.687     12.417  302220800
2012-01-04  14.643  14.810  14.617  14.766     12.484  260022000
2012-01-05  14.820  14.948  14.738  14.930     12.622  271269600
2012-01-06  14.992  15.098  14.972  15.086     12.754  318292800
2012-01-09  15.196  15.277  15.048  15.062     12.734  394024400


### From file

In [46]:
# read the Microsoft and Apple data from file
msft = pd.read_csv("msft.csv", index_col=0, parse_dates=True)
aapl = pd.read_csv("aapl.csv", index_col=0, parse_dates=True)

### Organizing the data for the examples

In [49]:
msft[:3]

             Open   High    Low  Close  Adj Close    Volume
Date                                                       
2012-01-03  26.55  26.96  26.39  26.77     21.201  64731500
2012-01-04  26.82  27.47  26.78  27.40     21.699  80516100
2012-01-05  27.38  27.73  27.29  27.68     21.921  56081400

In [51]:
aapl[:3]

              Open    High     Low   Close  Adj Close     Volume
Date                                                            
2012-01-03  14.621  14.732  14.607  14.687     12.417  302220800
2012-01-04  14.643  14.810  14.617  14.766     12.484  260022000
2012-01-05  14.820  14.948  14.738  14.930     12.622  271269600

# Reorganizing and reshaping data

## Concatenating data in Multiple DataFrame objects

In [57]:

# Get adjusted close data for Jan and Feb 2012 using loc
msftA01 = msft.loc['2012-01', ['Adj Close']]
msftA02 = msft.loc['2012-02', ['Adj Close']]

# Display the first three rows of January 2012
print(msftA01.head(3))

            Adj Close
Date                 
2012-01-03     21.201
2012-01-04     21.699
2012-01-05     21.921


In [59]:
msftA02[:3]

            Adj Close
Date                 
2012-02-01     23.671
2012-02-02     23.719
2012-02-03     23.949

In [61]:
# combine the first three rows of each of msftA01 and msftA02
pd.concat([msftA01.head(3), msftA02.head(3)])

            Adj Close
Date                 
2012-01-03     21.201
2012-01-04     21.699
2012-01-05     21.921
2012-02-01     23.671
2012-02-02     23.719
2012-02-03     23.949

In [67]:
# Extract only the Jan 2012 AAPL values using loc
aaplA01 = aapl.loc['2012-01', ['Adj Close']]

# Extract the Jan 2012 MSFT values using loc
msftA01 = msft.loc['2012-01', ['Adj Close']]

# Concatenate the AAPL and MSFT Jan 2012 data, there will be duplicate index labels
withDups = pd.concat([msftA01[:3], aaplA01[:3]], keys=['MSFT', 'AAPL'], names=['Stock', 'Date'])

print(withDups)

                  Adj Close
Stock Date                 
MSFT  2012-01-03     21.201
      2012-01-04     21.699
      2012-01-05     21.921
AAPL  2012-01-03     12.417
      2012-01-04     12.484
      2012-01-05     12.622


In [71]:
# Show the two records for the date 2012-01-03
records_2012_01_03 = withDups.loc[(slice(None), '2012-01-03'), :]
print(records_2012_01_03)

                  Adj Close
Stock Date                 
MSFT  2012-01-03     21.201
AAPL  2012-01-03     12.417


In [73]:
# demonstrate concat with a specification of the 
# stock tickets being part of the index
# this help disambiguate the duplicate dates using
# a hierarchal index
closes = pd.concat([msftA01[:3], aaplA01[:3]], 
                    keys=['MSFT', 'AAPL'])
closes

                 Adj Close
     Date                 
MSFT 2012-01-03     21.201
     2012-01-04     21.699
     2012-01-05     21.921
AAPL 2012-01-03     12.417
     2012-01-04     12.484
     2012-01-05     12.622

In [77]:
# Extract just MSFT values using loc and show the first three rows
msft_values = withDups.loc['MSFT'][:3]
print(msft_values)

            Adj Close
Date                 
2012-01-03     21.201
2012-01-04     21.699
2012-01-05     21.921


In [79]:
# demonstrate concatenation using two DataFrame's
# that each have two columns.  pandas will align the
# data in columns by the column names (labels)
msftAV = msft[['Adj Close', 'Volume']]
aaplAV = aapl[['Adj Close', 'Volume']]
pd.concat([msftAV, aaplAV])

            Adj Close     Volume
Date                            
2012-01-03     21.201   64731500
2012-01-04     21.699   80516100
2012-01-05     21.921   56081400
2012-01-06     22.262   99455500
...               ...        ...
2012-12-24     15.846  175753200
2012-12-26     15.627  302436400
2012-12-27     15.690  455120400
2012-12-28     15.524  354278400

[498 rows x 2 columns]

In [81]:
# demonstrate concatenation with DataFrame objects
# that do not have the same set of columns
# this demonstrates pandas filling in NaN values
aaplA = aapl[['Adj Close']]
pd.concat([msftAV, aaplA])

            Adj Close     Volume
Date                            
2012-01-03     21.201  6.473e+07
2012-01-04     21.699  8.052e+07
2012-01-05     21.921  5.608e+07
2012-01-06     22.262  9.946e+07
...               ...        ...
2012-12-24     15.846        NaN
2012-12-26     15.627        NaN
2012-12-27     15.690        NaN
2012-12-28     15.524        NaN

[498 rows x 2 columns]

In [83]:
# perform an inner join on the DataFrame's
# since aaplA does not have a Volume column, pandas
# will not include that column in the result
pd.concat([msftAV, aaplA], join='inner')

            Adj Close
Date                 
2012-01-03     21.201
2012-01-04     21.699
2012-01-05     21.921
2012-01-06     22.262
...               ...
2012-12-24     15.846
2012-12-26     15.627
2012-12-27     15.690
2012-12-28     15.524

[498 rows x 1 columns]

In [85]:
# concat along the rows, causing duplicate columns to
# be created in the result
msftA = msft[['Adj Close']]
closes = pd.concat([msftA, aaplA], axis=1)
closes[:3]

            Adj Close  Adj Close
Date                            
2012-01-03     21.201     12.417
2012-01-04     21.699     12.484
2012-01-05     21.921     12.622

In [87]:
# concat along rows using two DataFrame objects with
# different number of rows. This demonstrates how
# NaN values will be filled in those rows for AAPL
# which only hase three rows as compared to 5 for MSFT
pd.concat([msftAV[:5], aaplAV[:3]], axis=1,
          keys=['MSFT', 'AAPL'])

                MSFT                AAPL           
           Adj Close    Volume Adj Close     Volume
Date                                               
2012-01-03    21.201  64731500    12.417  3.022e+08
2012-01-04    21.699  80516100    12.484  2.600e+08
2012-01-05    21.921  56081400    12.622  2.713e+08
2012-01-06    22.262  99455500       NaN        NaN
2012-01-09    21.969  59706800       NaN        NaN

In [89]:
# inner join can also be used along this axis
# this will not include rows with index labels that do
# not exist in both DataFrame objects
pd.concat([msftA[:5], aaplA[:3]], axis=1,
          join='inner', keys=['MSFT', 'AAPL'])

                MSFT      AAPL
           Adj Close Adj Close
Date                          
2012-01-03    21.201    12.417
2012-01-04    21.699    12.484
2012-01-05    21.921    12.622

In [91]:
# ignore indexes and just concatenate the data and
# have the result have a default integer index
pd.concat([msftA[:3], aaplA[:3]], ignore_index=True)

   Adj Close
0     21.201
1     21.699
2     21.921
3     12.417
4     12.484
5     12.622

## Merging DataFrame objects

In [94]:
# we will merge these two DataFrame objects, 
# so lets peek at the data to remind ourselves
# of what they contain
msftAR = msftA.reset_index()
msftVR = msft[['Volume']].reset_index()
msftAR[:3]

        Date  Adj Close
0 2012-01-03     21.201
1 2012-01-04     21.699
2 2012-01-05     21.921

In [96]:
msftVR[:3]

        Date    Volume
0 2012-01-03  64731500
1 2012-01-04  80516100
2 2012-01-05  56081400

In [98]:
# merge the two.  pandas finds the columns in common,
# in this case Date, and merges on that column and adds
# a column for all the other columns in both DataFrame's
msftCVR = pd.merge(msftAR, msftVR)
msftCVR[:5]

        Date  Adj Close    Volume
0 2012-01-03     21.201  64731500
1 2012-01-04     21.699  80516100
2 2012-01-05     21.921  56081400
3 2012-01-06     22.262  99455500
4 2012-01-09     21.969  59706800

In [100]:
# we will demonstrate join semantics using this DataFrame
msftAR0_5 = msftAR[0:5]
msftAR0_5

        Date  Adj Close
0 2012-01-03     21.201
1 2012-01-04     21.699
2 2012-01-05     21.921
3 2012-01-06     22.262
4 2012-01-09     21.969

In [102]:
# and also this one
msftVR2_4 = msftVR[2:4]
msftVR2_4

        Date    Volume
2 2012-01-05  56081400
3 2012-01-06  99455500

In [104]:
# merge semantics using default inner join
pd.merge(msftAR0_5, msftVR2_4)

        Date  Adj Close    Volume
0 2012-01-05     21.921  56081400
1 2012-01-06     22.262  99455500

In [106]:
# same joing but using
pd.merge(msftAR0_5, msftVR2_4, how='outer')

        Date  Adj Close     Volume
0 2012-01-03     21.201        NaN
1 2012-01-04     21.699        NaN
2 2012-01-05     21.921  5.608e+07
3 2012-01-06     22.262  9.946e+07
4 2012-01-09     21.969        NaN

## Pivoting

In [109]:
# need to insert Symbol column before combining
msft.insert(0, 'Symbol', 'MSFT')
aapl.insert(0, 'Symbol', 'AAPL')

# concatenate the MSFT and AAPL data
# index will consist of the Date column, which we will sort
combined = pd.concat([msft, aapl]).sort_index()

# this pushes the index into a column and resets to a 
# default integer index
s4p = combined.reset_index();
s4p[:5]

        Date Symbol    Open    High     Low   Close  Adj Close     Volume
0 2012-01-03   MSFT  26.550  26.960  26.390  26.770     21.201   64731500
1 2012-01-03   AAPL  14.621  14.732  14.607  14.687     12.417  302220800
2 2012-01-04   MSFT  26.820  27.470  26.780  27.400     21.699   80516100
3 2012-01-04   AAPL  14.643  14.810  14.617  14.766     12.484  260022000
4 2012-01-05   MSFT  27.380  27.730  27.290  27.680     21.921   56081400

In [111]:
# pivot Date into the Index, make the columns match the
# unique values in the Symbol column, and the values 
# will be the AdjClose values
closes = s4p.pivot(index='Date', columns='Symbol', 
                   values='Adj Close')
closes[:3]

Symbol        AAPL    MSFT
Date                      
2012-01-03  12.417  21.201
2012-01-04  12.484  21.699
2012-01-05  12.622  21.921

## Stacking and Unstacking

In [114]:
# stack the first level of columns into the index
# essentially, moves AAPL and MSFT into the index
# leaving a single colum which is the AdjClose values
stackedCloses = closes.stack()
stackedCloses

Date        Symbol
2012-01-03  AAPL      12.417
            MSFT      21.201
2012-01-04  AAPL      12.484
            MSFT      21.699
                       ...  
2012-12-27  AAPL      15.690
            MSFT      21.955
2012-12-28  AAPL      15.524
            MSFT      21.621
Length: 498, dtype: float64

In [118]:
# Concatenate the AAPL and MSFT Jan 2012 data, there will be duplicate index labels
withDups = pd.concat([msftA01, aaplA01], keys=['MSFT', 'AAPL'], names=['Stock', 'Date'])

# To retrieve close values by specifying both the date and ticker
close_value = withDups.loc[('AAPL', '2012-01-03'), 'Adj Close']
print(close_value)

Stock  Date      
AAPL   2012-01-03    12.417
Name: Adj Close, dtype: float64


In [120]:
# Lookup on just the date, which will give us two values, one each for AAPL and MSFT
values_on_date = stackedCloses.loc[('2012-01-03')]
print(values_on_date)

Date        Symbol
2012-01-03  AAPL      12.417
            MSFT      21.201
dtype: float64


In [124]:
import yfinance as yf
import datetime
import pandas as pd

# Start and end dates
start = datetime.datetime(2012, 1, 1)
end = datetime.datetime(2012, 12, 30)

# Load the data
msft = yf.download("MSFT", start=start, end=end)
aapl = yf.download("AAPL", start=start, end=end)

# Extract only the Jan 2012 AAPL values using loc
aaplA01 = aapl.loc['2012-01', ['Adj Close']]

# Extract the Jan 2012 MSFT values using loc
msftA01 = msft.loc['2012-01', ['Adj Close']]

# Concatenate the AAPL and MSFT Jan 2012 data, there will be duplicate index labels
stackedCloses = pd.concat([msftA01, aaplA01], keys=['MSFT', 'AAPL'], names=['Stock', 'Date'])

# Look up all values for the MSFT symbol
msft_values = stackedCloses.loc['MSFT']
print(msft_values)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

            Adj Close
Date                 
2012-01-03     21.201
2012-01-04     21.699
2012-01-05     21.921
2012-01-06     22.262
...               ...
2012-01-26     23.363
2012-01-27     23.149
2012-01-30     23.450
2012-01-31     23.386

[20 rows x 1 columns]


In [126]:
# pivots the last level of the index back into a column
unstackedCloses = stackedCloses.unstack()
unstackedCloses[:3]

       Adj Close                                                         \
Date  2012-01-03 2012-01-04 2012-01-05 2012-01-06 2012-01-09 2012-01-10   
Stock                                                                     
MSFT      21.201     21.699     21.921     22.262     21.969     22.048   
AAPL      12.417     12.484     12.622     12.754     12.734     12.780   

                  ...                                                         \
Date  2012-01-11  ... 2012-01-23 2012-01-24 2012-01-25 2012-01-26 2012-01-27   
Stock             ...                                                          
MSFT      21.953  ...     23.545     23.236     23.410     23.363     23.149   
AAPL      12.759  ...     12.906     12.694     13.487     13.425     13.506   

                             
Date  2012-01-30 2012-01-31  
Stock                        
MSFT      23.450     23.386  
AAPL      13.679     13.783  

[2 rows x 20 columns]

## Melting

In [129]:
# melt making id_vars of Date and Symbol, making the 
# column names the variable and the for each the value
melted = pd.melt(s4p, id_vars=['Date', 'Symbol'])
melted[:5]

        Date Symbol variable   value
0 2012-01-03   MSFT     Open  26.550
1 2012-01-03   AAPL     Open  14.621
2 2012-01-04   MSFT     Open  26.820
3 2012-01-04   AAPL     Open  14.643
4 2012-01-05   MSFT     Open  27.380

In [131]:
# extract the values for the data for MSFT on 2012-01-03
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')]

           Date Symbol   variable      value
0    2012-01-03   MSFT       Open  2.655e+01
498  2012-01-03   MSFT       High  2.696e+01
996  2012-01-03   MSFT        Low  2.639e+01
1494 2012-01-03   MSFT      Close  2.677e+01
1992 2012-01-03   MSFT  Adj Close  2.120e+01
2490 2012-01-03   MSFT     Volume  6.473e+07

# Grouping and aggregation

## Splitting

In [135]:
# construct a DataFrame to demonstrate splitting
# extract from combined the Symbol and AdjClose, and reset the index
s4g = combined[['Symbol', 'Adj Close']].reset_index()
# now, add two columns, year and month, using the year and month
# portions of the data as integers
s4g.insert(1, 'Year', pd.DatetimeIndex(s4g['Date']).year)
s4g.insert(2, 'Month',pd.DatetimeIndex(s4g['Date']).month)
s4g[:5]

        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT     21.201
1 2012-01-03  2012      1   AAPL     12.417
2 2012-01-04  2012      1   MSFT     21.699
3 2012-01-04  2012      1   AAPL     12.484
4 2012-01-05  2012      1   MSFT     21.921

In [137]:
# group by the Symbol column
s4g.groupby('Symbol')

In [139]:
# group again, but save the result this time
grouped = s4g.groupby('Symbol')
# the groupby object has a property groups, which shows how
# all rows will in mapped into the groups.   
# the type of this object is a python dict
type(grouped.groups)

pandas.io.formats.printing.PrettyDict

In [141]:
# show the mappings of rows to groups
grouped.groups

{'AAPL': [1, 3, 5, 7, 9, 11, 13, 14, 16, 18, 20, 23, 25, 27, 29, 30, 33, 34, 37, 38, 41, 43, 44, 46, 49, 51, 52, 54, 57, 59, 61, 62, 64, 67, 68, 71, 72, 75, 77, 79, 81, 82, 84, 86, 89, 90, 93, 95, 96, 99, 100, 103, 105, 107, 109, 111, 112, 114, 117, 119, 121, 122, 124, 126, 128, 130, 132, 134, 137, 139, 140, 142, 145, 147, 149, 151, 153, 154, 156, 158, 160, 163, 164, 166, 168, 170, 173, 175, 176, 178, 180, 183, 184, 186, 189, 190, 193, 194, 196, 198, ...], 'MSFT': [0, 2, 4, 6, 8, 10, 12, 15, 17, 19, 21, 22, 24, 26, 28, 31, 32, 35, 36, 39, 40, 42, 45, 47, 48, 50, 53, 55, 56, 58, 60, 63, 65, 66, 69, 70, 73, 74, 76, 78, 80, 83, 85, 87, 88, 91, 92, 94, 97, 98, 101, 102, 104, 106, 108, 110, 113, 115, 116, 118, 120, 123, 125, 127, 129, 131, 133, 135, 136, 138, 141, 143, 144, 146, 148, 150, 152, 155, 157, 159, 161, 162, 165, 167, 169, 171, 172, 174, 177, 179, 181, 182, 185, 187, 188, 191, 192, 195, 197, 199, ...]}

In [143]:
# these report the number of groups that resulted from
# the grouping
len(grouped), grouped.ngroups

(2, 2)

In [151]:
# this function will print the contents of a group
def print_groups (groupobject):
    for name, group in groupobject:
        print (name)
        print (group.head())

In [153]:
# examine our resulting groups
print_groups(grouped)

AAPL
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL     12.417
3 2012-01-04  2012      1   AAPL     12.484
5 2012-01-05  2012      1   AAPL     12.622
7 2012-01-06  2012      1   AAPL     12.754
9 2012-01-09  2012      1   AAPL     12.734
MSFT
        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT     21.201
2 2012-01-04  2012      1   MSFT     21.699
4 2012-01-05  2012      1   MSFT     21.921
6 2012-01-06  2012      1   MSFT     22.262
8 2012-01-09  2012      1   MSFT     21.969


In [155]:
# .size will tell us the count of items in each group
grouped.size()

Symbol
AAPL    249
MSFT    249
dtype: int64

In [157]:
# a specific group can be retrieved using .get_group()
# which returns a DataFrame representing the specified group
grouped.get_group('MSFT')

          Date  Year  Month Symbol  Adj Close
0   2012-01-03  2012      1   MSFT     21.201
2   2012-01-04  2012      1   MSFT     21.699
4   2012-01-05  2012      1   MSFT     21.921
6   2012-01-06  2012      1   MSFT     22.262
..         ...   ...    ...    ...        ...
491 2012-12-24  2012     12   MSFT     22.036
493 2012-12-26  2012     12   MSFT     21.873
495 2012-12-27  2012     12   MSFT     21.955
496 2012-12-28  2012     12   MSFT     21.621

[249 rows x 5 columns]

In [159]:
# group by three different fields and print the result
mcg = s4g.groupby(['Symbol', 'Year', 'Month'])
print_groups(mcg)

('AAPL', 2012, 1)
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL     12.417
3 2012-01-04  2012      1   AAPL     12.484
5 2012-01-05  2012      1   AAPL     12.622
7 2012-01-06  2012      1   AAPL     12.754
9 2012-01-09  2012      1   AAPL     12.734
('AAPL', 2012, 2)
         Date  Year  Month Symbol  Adj Close
41 2012-02-01  2012      2   AAPL     13.775
43 2012-02-02  2012      2   AAPL     13.742
44 2012-02-03  2012      2   AAPL     13.880
46 2012-02-06  2012      2   AAPL     14.009
49 2012-02-07  2012      2   AAPL     14.156
('AAPL', 2012, 3)
         Date  Year  Month Symbol  Adj Close
81 2012-03-01  2012      3   AAPL     16.440
82 2012-03-02  2012      3   AAPL     16.462
84 2012-03-05  2012      3   AAPL     16.099
86 2012-03-06  2012      3   AAPL     16.011
89 2012-03-07  2012      3   AAPL     16.024
('AAPL', 2012, 4)
          Date  Year  Month Symbol  Adj Close
124 2012-04-02  2012      4   AAPL     18.679
126 2012-04-03  2012      4   AA


('MSFT', 2012, 10)
          Date  Year  Month Symbol  Adj Close
376 2012-10-01  2012     10   MSFT      27.44
378 2012-10-02  2012     10   MSFT      27.60
381 2012-10-03  2012     10   MSFT      27.78
383 2012-10-04  2012     10   MSFT      27.94
384 2012-10-05  2012     10   MSFT      27.77
('MSFT', 2012, 11)
          Date  Year  Month Symbol  Adj Close
418 2012-11-01  2012     11   MSFT      27.47
421 2012-11-02  2012     11   MSFT      27.45
422 2012-11-05  2012     11   MSFT      27.57
424 2012-11-06  2012     11   MSFT      27.78
426 2012-11-07  2012     11   MSFT      27.06
('MSFT', 2012, 12)
          Date  Year  Month Symbol  Adj Close
461 2012-12-03  2012     12   MSFT      24.79
462 2012-12-04  2012     12   MSFT      24.74
465 2012-12-05  2012     12   MSFT      25.02
467 2012-12-06  2012     12   MSFT      25.07
469 2012-12-07  2012     12   MSFT      24.82


In [161]:
# set the index of the data to be the following three fields
# we are creating a multiindex
mi = s4g.set_index(['Symbol', 'Year', 'Month'])
mi

                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03     21.201
AAPL   2012 1     2012-01-03     12.417
MSFT   2012 1     2012-01-04     21.699
AAPL   2012 1     2012-01-04     12.484
...                      ...        ...
            12    2012-12-27     15.690
MSFT   2012 12    2012-12-27     21.955
            12    2012-12-28     21.621
AAPL   2012 12    2012-12-28     15.524

[498 rows x 2 columns]

In [163]:
# now we can group based upon values in the actual index
# the following groups by level 0 of the index (Month)
mig_l1 = mi.groupby(level=0)
print_groups(mig_l1)

AAPL
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03     12.417
            1     2012-01-04     12.484
            1     2012-01-05     12.622
            1     2012-01-06     12.754
            1     2012-01-09     12.734
MSFT
                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03     21.201
            1     2012-01-04     21.699
            1     2012-01-05     21.921
            1     2012-01-06     22.262
            1     2012-01-09     21.969


In [165]:
# group by three levels in the index using their names
mig_l12 = mi.groupby(level=['Symbol', 'Year', 'Month'])
print_groups(mig_l12)

('AAPL', 2012, 1)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03     12.417
            1     2012-01-04     12.484
            1     2012-01-05     12.622
            1     2012-01-06     12.754
            1     2012-01-09     12.734
('AAPL', 2012, 2)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 2     2012-02-01     13.775
            2     2012-02-02     13.742
            2     2012-02-03     13.880
            2     2012-02-06     14.009
            2     2012-02-07     14.156
('AAPL', 2012, 3)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 3     2012-03-01     16.440
            3     2012-03-02     16.462
            3     2012-03-05     16.099
            3     2012-03-06     16.011
            3     2012-03-07     16.024
('AAPL', 2012, 4)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2

# Aggregation

In [168]:
# this will apply the mean function to each group
mig_l12.agg(np.mean)

                   Adj Close
Symbol Year Month           
AAPL   2012 1         12.941
            2         15.024
            3         17.438
            4         18.298
...                      ...
MSFT   2012 9         24.863
            10        23.477
            11        22.575
            12        21.971

[24 rows x 1 columns]

In [170]:
# example of groupby that also ignores the index
# resulting in a default integer index
# this also has the mean function applied
s4g.groupby(['Symbol', 'Year', 'Month'], 
            as_index=False).agg(np.mean)[:5]

  Symbol  Year  Month  Adj Close
0   AAPL  2012      1     12.941
1   AAPL  2012      2     15.024
2   AAPL  2012      3     17.438
3   AAPL  2012      4     18.298
4   AAPL  2012      5     17.050

In [172]:
# apply multiple functions to each group in one call
mig_l12.agg([np.mean, np.std])

                  Adj Close       
                       mean    std
Symbol Year Month                 
AAPL   2012 1        12.941  0.403
            2        15.024  0.800
            3        17.438  0.933
            4        18.298  0.686
...                     ...    ...
MSFT   2012 9        24.863  0.374
            10       23.477  0.579
            11       22.575  0.869
            12       21.971  0.316

[24 rows x 2 columns]